In [1]:
import numpy as np
import sklearn   
import os, sys
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
import numpy as np
from sklearn.model_selection import StratifiedKFold
#参数配置模块
from easydict import EasyDict as edict
config = edict()
config.TRAINDATA='KDDTrain_20Percent.csv'
config.TESTDATA='KDDTest.csv'
# config.VALIDATION_SPLIT = 0.5  # 整个 *样本集* 划分给测试集的大小
# correspondence = "./data/training_attack_types.txt"

In [2]:
data = pd.read_csv(config.TRAINDATA, header=None)
data1 = pd.read_csv(config.TESTDATA, header=None)
data1.shape

(22544, 43)

In [8]:
# 归一化
from sklearn.preprocessing import MinMaxScaler
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
data[data.columns.difference([1,2,3,41,42])].apply(max_min_scaler)
data1[data1.columns.difference([1,2,3,41,42])].apply(max_min_scaler)

,0,4,5,6,7,8,9,10,11,12,...,31,32,33,34,35,36,37,38,39,40
0,0.000035,2.066513e-04,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.525490,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00
1,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,1.000000,0.039216,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00
2,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,1.000000,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00
3,0.000000,3.183413e-07,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.011765,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00
4,0.000017,0.000000e+00,0.000011,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.113725,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0.000000,1.263815e-05,0.000247,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.392157,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00
22540,0.000000,5.045710e-06,0.000697,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.772549,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00
22541,0.000000,8.681168e-04,0.006177,0.0,0.0,0.0,0.019802,0.0,1.0,0.001256,...,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07
22542,0.000000,6.685168e-07,0.000031,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,1.000000,0.988235,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00


 将标签转换为两类

In [9]:
for i in range(data.shape[0]):
    if data.iat[i, 41] !=0:
        data.iat[i, 41] = 1
for i in range(data1.shape[0]):
    if data1.iat[i, 41] !=0:
        data1.iat[i, 41] = 1

In [10]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
# resource = mm.fit_transform(np.array(data[data.columns.difference([41])]))
# X_train = mm.fit_transform(np.array(data[data.columns.difference([41])]))
X_train = np.array(data[data.columns.difference([41,42])])
y_train=np.array(data[41])
# X_test = mm.fit_transform(np.array(data1[data1.columns.difference([41])]))
X_test = np.array(data1[data1.columns.difference([41,42])])
y_test=np.array(data1[41])
#切分训练集和测试集
# from sklearn.model_selection import train_test_split
# labels=np.array(data[41])
# X_train, X_test, y_train, y_test = train_test_split(resource, labels, test_size=config.VALIDATION_SPLIT,random_state=42)

In [11]:
# 评估
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
def getScore(y_test,y_pred):
#     print("Precision", precision_score(y_test, y_pred))
#     print("recall_score", recall_score(y_test, y_pred))
    print("Accuracy:"+str(accuracy_score(y_test,y_pred)))
    print("confusion_matrix", confusion_matrix(y_test, y_pred))
    c=confusion_matrix(y_test, y_pred)
    recall=c[0][0]/(c[0][0]+c[0][1])
    precision=c[0][0]/(c[0][0]+c[1][0])
    f1=(2*recall*precision)/(recall+precision)
    print("Recall:"+str(recall))
    print("Precision:"+str(precision))
    print("F-M", str(f1))
    print("original FPR:"+str(c[1][0]/(c[1][0]+c[1][1])))
    print("original AUC:"+str(roc_auc_score(y_test,y_pred)))

融合五种类型的特征C(4,2)

In [32]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
start_time = time.time()  # 记录程序开始运行时间
#0-8 #9-21 #22-30 #31-40
pipe1 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21)),
                      DecisionTreeClassifier())
pipe2 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,22,23,24,25,26,27,28,29,30)),
                      DecisionTreeClassifier())
pipe3 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
pipe4 = make_pipeline(ColumnSelector(cols=(9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30)),
                      DecisionTreeClassifier())
pipe5 = make_pipeline(ColumnSelector(cols=(9,10,11,12,13,14,15,16,17,18,19,20,21,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
pipe6 = make_pipeline(ColumnSelector(cols=(22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
sclf = StackingClassifier(classifiers=[pipe1, pipe2,pipe3, pipe4,pipe5, pipe6], use_probas=True,
                         average_probas=True,meta_classifier=RandomForestClassifier())
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)
getScore(y_test,y_pred)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))

Accuracy:0.8007452093683464
confusion_matrix [[9066  645]
 [3847 8986]]
Recall:0.9335804757491505
Precision:0.7020831719972122
F-M 0.8014497878359265
original FPR:0.2997740201044183
original AUC:0.8169032278223661
Took 1.390281 second


融合五种类型的特征C(4,3)

In [29]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
start_time = time.time()  # 记录程序开始运行时间
#0-8 #9-21 #22-30 #31-40
pipe1 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                           18,19,20,21,22,23,24,25,26,27,28,29,30)),
                      DecisionTreeClassifier())
pipe2 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                           18,19,20,21,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
pipe3 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,22,23,24,25,26,27,28,29,
                                           30,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
pipe4 = make_pipeline(ColumnSelector(cols=(9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
                                           25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
sclf = StackingClassifier(classifiers=[pipe1, pipe2,pipe3, pipe4], use_probas=True,
                          average_probas=True,meta_classifier=RandomForestClassifier())
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)
getScore(y_test,y_pred)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))

Accuracy:0.8402679205110007
confusion_matrix [[9361  350]
 [3251 9582]]
Recall:0.9639583976933375
Precision:0.7422296225816682
F-M 0.8386865564664249
original FPR:0.253331255357282
original AUC:0.8553135711680278
Took 1.326452 second


融合五种类型的特征C(4,1)

In [13]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
start_time = time.time()  # 记录程序开始运行时间
#0-8 #9-21 #22-30 #31-40
pipe1 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8)),
                      DecisionTreeClassifier())
pipe2 = make_pipeline(ColumnSelector(cols=(9,10,11,12,13,14,15,16,17,18,19,20,21)),
                      DecisionTreeClassifier())
pipe3 = make_pipeline(ColumnSelector(cols=(22,23,24,25,26,27,28,29,30)),
                      DecisionTreeClassifier())
pipe4 = make_pipeline(ColumnSelector(cols=(31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
sclf = StackingClassifier(classifiers=[pipe1, pipe2,pipe3, pipe4], use_probas=True,
                          average_probas=True,meta_classifier=RandomForestClassifier())
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)
getScore(y_test,y_pred)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))

Accuracy:0.7659244144783535
confusion_matrix [[9112  599]
 [4678 8155]]
Recall:0.9383173720523118
Precision:0.6607686729514141
F-M 0.7754563635589975
original FPR:0.36452894880386505
original AUC:0.7868942116242235
Took 1.326298 second


融合五种类型的特征C(4,4)

In [49]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
start_time = time.time()  # 记录程序开始运行时间
#0-8 #9-21 #22-30 #31-40
pipe1 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,
                                           18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
sclf = StackingClassifier(classifiers=[pipe1], use_probas=True,
                          average_probas=True,meta_classifier=RandomForestClassifier())
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)
getScore(y_test,y_pred)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))

Accuracy:0.8159155429382541
confusion_matrix [[9392  319]
 [3831 9002]]
Recall:0.9671506538976419
Precision:0.7102775466989337
F-M 0.8190459579663382
original FPR:0.29852723447362267
original AUC:0.8343117097120096
Took 0.979432 second


简单地分前后各一半特征

In [15]:
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
pipe1 = make_pipeline(ColumnSelector(cols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)),
                      DecisionTreeClassifier())
pipe2 = make_pipeline(ColumnSelector(cols=(22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40)),
                      DecisionTreeClassifier())
sclf = StackingClassifier(classifiers=[pipe1, pipe2], use_probas=True,
                          average_probas=True,meta_classifier=RandomForestClassifier())
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)
getScore(y_test,y_pred)

Accuracy:0.7643275372604684
confusion_matrix [[9471  240]
 [5073 7760]]
Recall:0.9752857584182886
Precision:0.6511963696369637
F-M 0.780952380952381
original FPR:0.39530896906413154
original AUC:0.7899883946770785


单个分类器

In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
import time
clfs = [RandomForestClassifier(),ExtraTreesClassifier(),DecisionTreeClassifier(),LGBMClassifier(),GaussianNB(),MultinomialNB(),KNeighborsClassifier()]
tips=['RandomForestClassifier','ExtraTreesClassifier','DecisionTreeClassifier','LGBMClassifier','GaussianNB','MultinomialNB','KNeighborsClassifier']
for tip,clf in zip(tips,clfs):
    print(tip)
    start_time = time.time()  # 记录程序开始运行时间
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    getScore(y_test,y_pred) 
    end_time = time.time()  # 记录程序结束运行时间
    print('Took %f second' % (end_time - start_time))

RandomForestClassifier
Accuracy:0.772888573456352
confusion_matrix [[9441  270]
 [4850 7983]]
Recall:0.9721964782205746
Precision:0.6606255685396404
F-M 0.7866844429630864
original FPR:0.3779318943349178
original AUC:0.7971322919428283
Took 2.529233 second
ExtraTreesClassifier
Accuracy:0.763972675656494
confusion_matrix [[9424  287]
 [5034 7799]]
Recall:0.9704458861085367
Precision:0.6518190621109421
F-M 0.7798419462948405
original FPR:0.39226992908906727
original AUC:0.7890879785097348
Took 2.111394 second
DecisionTreeClassifier
Accuracy:0.8144517388218595
confusion_matrix [[9400  311]
 [3872 8961]]
Recall:0.9679744619503655
Precision:0.7082579867389994
F-M 0.8179959100204499
original FPR:0.30172212265253645
original AUC:0.8331261696489145
Took 0.233296 second
LGBMClassifier
Accuracy:0.7824254790631654
confusion_matrix [[9420  291]
 [4614 8219]]
Recall:0.9700339820821748
Precision:0.6712270200940573
F-M 0.7934301958307012
original FPR:0.3595418062806826
original AUC:0.805246087900746


投票法

In [73]:
from sklearn.ensemble import VotingClassifier
start_time = time.time()  # 记录程序开始运行时间
clfs = [
        ('DecisionTreeClassifier', DecisionTreeClassifier()),
        ('RandomForestClassifier',RandomForestClassifier())
    ]
clf = VotingClassifier(estimators=clfs, voting='soft')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
getScore(y_test,y_pred)
end_time = time.time()  # 记录程序结束运行时间
print('Took %f second' % (end_time - start_time))

Accuracy:0.814540454222853
confusion_matrix [[9399  312]
 [3869 8964]]
Recall:0.9678714859437751
Precision:0.7083961410913476
F-M 0.8180512641977458
original FPR:0.30148835034676225
original AUC:0.8331915677985066
Took 2.619956 second
